In [0]:
%%sh
# Download the CMS Open Payments ZIP file
curl -L https://download.cms.gov/openpayments/PGYR2023_P01302025_01212025.zip -o /tmp/openpayments_2023.zip

# Unzip the file to a directory, force overwriting existing files without prompting
unzip -o /tmp/openpayments_2023.zip -d /tmp/openpayments_2023

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  752M  100  752M    0     0  40.2M      0  0:00:18  0:00:18 --:--:-- 47.1M


Archive:  /tmp/openpayments_2023.zip
  inflating: /tmp/openpayments_2023/OP_PGYR2023_README_P01302025.txt  
  inflating: /tmp/openpayments_2023/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv  


In [0]:
# Importing required libraries
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc, count

In [0]:
import os

# List the extracted files
os.listdir("/tmp/openpayments_2023")

Out[3]: ['OP_PGYR2023_README_P01302025.txt',
 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv']

In [0]:
# Reading the CSV file into a Spark DataFrame
df = spark.read.csv("file:/tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)

# Saving it as a Delta table
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("asritha_week7_General_openPays_2023")

In [0]:
# Reading the CSV file into a Spark DataFrame
df = spark.read.csv("dbfs:/FileStore/shared_uploads/asritha.suraparaju@slu.edu/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025-1.csv", header=True, inferSchema=True)

# Saving it as a Delta table (use underscores instead of spaces)
df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("Cp_covered_recipients")


In [0]:
from pyspark.sql.functions import col
# 1. Nature of Payments with reimbursement amounts greater than $1,000, ordered by count
df_general = spark.read.table("asritha_week7_General_openPays_2023")

df_general.filter(col("Total_Amount_of_Payment_USDollars") > 1000) \
    .groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10, False)


+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count |
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|164092|
|Consulting Fee                                                                                                                                    |105228|
|Travel and Lodging                                                                                                                                |24738 |
|Honoraria                                                      

In [0]:

#Top 10 Nature of Payments by count
df_general.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10, False)




+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |13378464|
|Travel and Lodging                                                                                                                                |545086  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|236628  |
|Consulting Fee                                     

In [0]:
#Top 10 Nature of Payments by total amount
from pyspark.sql.functions import sum

df_general.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("Total_Payment")) \
    .orderBy(col("Total_Payment").desc()) \
    .show(10, False)


+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |Total_Payment       |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Royalty or License                                                                                                                                |1.1921745630200038E9|
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|5.946326876500018E8 |
|Consulting Fee                                                                                                                                    |5.

In [0]:
df_joined = df_general.join(df_recipients, "Covered_Recipient_Profile_ID", "left")

# Now, group by `Physician_Specialty` from the recipients table and aggregate payments
df_joined.groupBy("Covered_Recipient_Profile_Primary_Specialty") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("Total_Payment")) \
    .orderBy(col("Total_Payment").desc()) \
    .show(10, False)



+------------------------------------------------------------------------------------------------+-------------------+
|Covered_Recipient_Profile_Primary_Specialty                                                     |Total_Payment      |
+------------------------------------------------------------------------------------------------+-------------------+
|null                                                                                            |8.712034564900051E8|
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                         |3.777555961599969E8|
|Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology                            |8.680715037000023E7|
|Allopathic & Osteopathic Physicians|Neurological Surgery                                        |8.58704690299999E7 |
|Allopathic & Osteopathic Physicians|Dermatology                                                 |8.076589144000082E7|
|Allopathic & Osteopathic Physicians|Orthopaedic

In [0]:
# Join the two datasets (General Payments and Covered Recipients)
df_joined = df_general.join(df_recipients, "Covered_Recipient_Profile_ID", "left")

# Group by the physician's first and last name (or use NPI if you prefer) and sum the total payment amounts
df_joined.groupBy("Covered_Recipient_First_Name", "Covered_Recipient_Last_Name") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("Total_Payment")) \
    .orderBy(col("Total_Payment").desc()) \
    .show(10, False)


+----------------------------+---------------------------+-------------------+
|Covered_Recipient_First_Name|Covered_Recipient_Last_Name|Total_Payment      |
+----------------------------+---------------------------+-------------------+
|null                        |null                       |7.933900857900007E8|
|STEPHEN                     |BURKHART                   |3.392202493E7      |
|WILLIAM                     |BINDER                     |2.943437497E7      |
|KEVIN                       |FOLEY                      |1.73059378E7       |
|IVAN                        |OSORIO                     |1.606551551E7      |
|GEORGE                      |MAXWELL                    |1.160032024E7      |
|ROBERT                      |BOOTH                      |8459167.19         |
|NEAL                        |ELATTRACHE                 |7810628.200000001  |
|AARON                       |ROSENBERG                  |6883627.29         |
|ROGER                       |JACKSON               